In [18]:
# 用kaggle手写数字识别数据

import pandas as pd
import numpy as np

train = np.matrix(pd.read_csv("dr_train.csv"))
train_x = train[:, 1:]
train_y = train[:, 0].A1

test_x = np.matrix(pd.read_csv("dr_test.csv"))

In [19]:
from sklearn import tree
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10)

"""
for depth in range(3, 15):
    scores = []
    for idx_train, idx_test in kfold.split(train_x, train_y):
        trn_x = train_x[idx_train]
        trn_y = train_y[idx_train]
        trn_tst_x = train_x[idx_test]
        trn_tst_y = train_y[idx_test]
        clf = tree.DecisionTreeClassifier(max_depth=depth)
        clf.fit(trn_x, trn_y)
        scores.append(clf.score(trn_tst_x, trn_tst_y))
    print(depth, np.mean(scores))
"""

def output_csv(filename, y):
    df_pred = pd.DataFrame({"ImageId": range(1, len(y) + 1), "Label": y}) 
    df_pred.to_csv(filename, index=False)

def _choose(pred):
    """ Choose the most    """
    m = {}
    for n in pred:
        m[n] = m.get(n, 0) + 1
    return sorted(m.keys(), key=lambda n:m[n])[-1]

def bulk_predict(trained_models, x_test):
    predicts = []
    for m in trained_models:
        y_predict = m.predict(x_test)
        predicts.append(y_predict)
    pred_mat = np.matrix(predicts)
    y = []
    for i in range(pred_mat.shape[1]):
        p = _choose(pred_mat[:, i].A1)
        y.append(p)
    return np.array(y)

clf = tree.DecisionTreeClassifier(max_depth=7)
dt_models = []
for idx_train, idx_test in kfold.split(train_x, train_y):
    trn_x = train_x[idx_train]
    trn_y = train_y[idx_train]
    trn_tst_x = train_x[idx_test]
    trn_tst_y = train_y[idx_test]
    clf = tree.DecisionTreeClassifier(max_depth=7)
    clf.fit(trn_x, trn_y)
    print(clf.score(trn_tst_x, trn_tst_y))
    dt_models.append(clf)
y_test = bulk_predict(dt_models, test_x)
output_csv("dt_kaggle.csv", y_test)

0.786019971469
0.77973358706
0.774631128034
0.773863365865
0.782142857143
0.772803048345
0.77399380805
0.773647843698
0.781935176358
0.779075309819
